In [5]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Cargar el archivo .xlsx
file_path = 'FGR_data_para_proyecto_final_clean_organizado.xlsx'  # Asegúrate de que esta ruta sea correcta
df = pd.read_excel(file_path)

# Eliminar columnas completamente vacías
df = df.dropna(axis=1, how='all')

# Separar las características (X) y la variable objetivo (y)
X = df.iloc[:, :-1]  # Todas las columnas excepto la última
y = df.iloc[:, -1]   # La última columna (Fetal weight)

# Imputar los valores NaN
imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Aplicar SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_imputed, y)

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, stratify=y_res, random_state=42)

# Combinar X_train con y_train y X_test con y_test
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

def save_as_arff(df, filename):
    with open(filename, 'w') as f:
        f.write('@relation dataset_diabetes_for_fcm_expert\n\n')

        for i, col in enumerate(df.columns[:-1]):
            f.write(f'@attribute c{i+1} real\n')

        f.write('@attribute class {0.0, 1.0}\n')
        f.write('\n@data\n')

        for index, row in df.iterrows():
            class_value = 1.0 if row.iloc[-1] > 0 else 0.0
            data_row = row[:-1].values.tolist() + [class_value]
            f.write(','.join(map(str, data_row)) + '\n')

# Guardar los archivos ARFF
save_as_arff(train_data, 'train.arff')
save_as_arff(test_data, 'test.arff')

# Opcional: Guardar también el dataset completo balanceado
full_data = pd.concat([X_res, y_res], axis=1)
save_as_arff(full_data, 'dataset.arff')

print("Archivos ARFF generados exitosamente.")

# Opcional: Mostrar las distribuciones de clases
print("\nDistribución de clases en el dataset original:")
print(y.value_counts())

print("\nDistribución de clases después de SMOTE:")
print(y_res.value_counts())

print("\nDistribución de clases en el conjunto de entrenamiento:")
print(y_train.value_counts())

print("\nDistribución de clases en el conjunto de prueba:")
print(y_test.value_counts())

Archivos ARFF generados exitosamente.

Distribución de clases en el dataset original:
Fetal weight
0    56
1    16
Name: count, dtype: int64

Distribución de clases después de SMOTE:
Fetal weight
0    56
1    56
Name: count, dtype: int64

Distribución de clases en el conjunto de entrenamiento:
Fetal weight
0    39
1    39
Name: count, dtype: int64

Distribución de clases en el conjunto de prueba:
Fetal weight
1    17
0    17
Name: count, dtype: int64
